# Images downloading methods




Following are the ways I used to download images from internet for data analysis purpose
*   Using ```search_image_ddg```
*   Using [Pyimage search method](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)


## search_image_ddg method

From fastbook webiste you can see the documentaion, however here I reproduce code snippet for ease of use.

In [ ]:
!pip install fastbook -Uqq
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [ ]:
from fastbook import *

In [ ]:
def search_images_ddg(term, max_images=200): # search_image_ddg in fastbook version has some issues so defining seperately
    "Search for `term` with DuckDuckGo and return a unique urls of about `max_images` images"
    assert max_images<1000
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    assert searchObj
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    urls,data = set(),{'next':1}
    while len(urls)<max_images and 'next' in data:
        try:
            data = urljson(requestUrl,data=params)
            urls.update(L(data['results']).itemgot('image'))
            requestUrl = url + data['next']
        except (URLError,HTTPError): pass
        time.sleep(0.2)
    return L(urls)

In [ ]:
results = search_images_ddg('search word', max_images=300)

In [ ]:
download_images(dest, urls=results) #destination folder path

## Another more elegant method using jmd_imagescraper (Thanks to Joe Dockrill)

Recently I found `jmd_imagescraper` library from Joe Dockrill. It is lot easier to get images data from web using DuckDuckGo search engine, better than previous method shown.

Follwoing is the code snippet for downloading any images:


In [ ]:
!pip install jmd_imagescraper -Uqq
from jmd_imagescraper.core import *
from pathlib import Path

root = Path().cwd()/'folder_name'

search = duckduckgo_search

search(root, 'prez-biden','joe biden ', max_results = 50, img_layout=ImgLayout.All)

In [3]:
from jmd_imagescraper.imagecleaner import *
display_image_cleaner(root)

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…

Using above method we can clean our dataset by deleting whichever is not according to the description of the dataset. using this library we can clearly structure the dataset with proper folder structure.

## Using pyimage search method

Pyimage search is well known for computer vision tutorials and Mr Adrian Rosebrock has given code snippet in his blog [here](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)

You only need to copy the following code snippet into the Javascript console and type enter. Text file will be downloaded comprising all urls related to the searched images. javascript console will be opened in browser by pressing ```F12``` key in either mozilla firefox or chrome.

In [ ]:
function simulateRightClick( element ) {
    var event1 = new MouseEvent( 'mousedown', {
        bubbles: true,
        cancelable: false,
        view: window,
        button: 2,
        buttons: 2,
        clientX: element.getBoundingClientRect().x,
        clientY: element.getBoundingClientRect().y
    } );
    element.dispatchEvent( event1 );
    var event2 = new MouseEvent( 'mouseup', {
        bubbles: true,
        cancelable: false,
        view: window,
        button: 2,
        buttons: 0,
        clientX: element.getBoundingClientRect().x,
        clientY: element.getBoundingClientRect().y
    } );
    element.dispatchEvent( event2 );
    var event3 = new MouseEvent( 'contextmenu', {
        bubbles: true,
        cancelable: false,
        view: window,
        button: 2,
        buttons: 0,
        clientX: element.getBoundingClientRect().x,
        clientY: element.getBoundingClientRect().y
    } );
    element.dispatchEvent( event3 );
}

function getURLParam( queryString, key ) {
    var vars = queryString.replace( /^\?/, '' ).split( '&' );
    for ( let i = 0; i < vars.length; i++ ) {
        let pair = vars[ i ].split( '=' );
        if ( pair[0] == key ) {
            return pair[1];
        }
    }
    return false;
}

function createDownload( contents ) {
    var hiddenElement = document.createElement( 'a' );
    hiddenElement.href = 'data:attachment/text,' + encodeURI( contents );
    hiddenElement.target = '_blank';
    hiddenElement.download = 'urls.txt';
    hiddenElement.click();
}

function grabUrls() {
    var urls = [];
    return new Promise( function( resolve, reject ) {
        var count = document.querySelectorAll(
        	'.isv-r a:first-of-type' ).length,
            index = 0;
        Array.prototype.forEach.call( document.querySelectorAll(
        	'.isv-r a:first-of-type' ), function( element ) {
            // using the right click menu Google will generate the
            // full-size URL; won't work in Internet Explorer
            // (http://pyimg.co/byukr)
            simulateRightClick( element.querySelector( ':scope img' ) );
            // Wait for it to appear on the <a> element
            var interval = setInterval( function() {
                if ( element.href.trim() !== '' ) {
                    clearInterval( interval );
                    // extract the full-size version of the image
                    let googleUrl = element.href.replace( /.*(\?)/, '$1' ),
                        fullImageUrl = decodeURIComponent(
                        	getURLParam( googleUrl, 'imgurl' ) );
                    if ( fullImageUrl !== 'false' ) {
                        urls.push( fullImageUrl );
                    }
                    // sometimes the URL returns a "false" string and
                    // we still want to count those so our Promise
                    // resolves
                    index++;
                    if ( index == ( count - 1 ) ) {
                        resolve( urls );
                    }
                }
            }, 10 );
        } );
    } );
}

grabUrls().then( function( urls ) {
    urls = urls.join( '\n' );
    createDownload( urls );
} );

After downloading the urls text file, upload to the jupyter notebook or download the images just as before


In [ ]:
download_images(dest='./file', urls=Path('/content/urls.txt'))

## storing the collected images

Downloaded files from the colab can be stored using the following code snippet

In [ ]:
# collect the files
zip_name = 'folder_name.zip'
!rm -f {zip_name}
!zip -q -r {zip_name} {root}

# download the files to your local computer
from google.colab import files
files.download(zip_name)

# download the files to Google Drive
from google.colab import drive
import shutil
destination_folder = 'folder_name'
drive.mount('/content/drive/')
folder = Path('/content/drive/My Drive')/destination_folder
folder.mkdir(parents=True, exist_ok = True)
shutil.copyfile(zip_name, str(folder/zip_name))

# open the collected files
!unzip \*zip && rm *.zip

### credits
1. [Maria L Rodriguez](https://medium.com/artificialis/convolutional-neural-network-backbone-c584ef162392)
2. [joeDockrill](https://github.com/joedockrill/jmd_imagescraper)
3. [pyimagesearch](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)